In [233]:
import xml.etree.ElementTree as ET
import pandas as pd


# parse export.xml
tree = ET.parse('export.xml')
root = tree.getroot()
# only get records of workouts
record_list = []
for workout_record in root.iter('Workout'):
    metadata = workout_record.findall('MetadataEntry')
    statistics = workout_record.findall('WorkoutStatistics')
    additional_stats = {}
    
    for metadata_entry in metadata:
        if metadata_entry.attrib['key'] == 'HKTimeZone':
            additional_stats['timeZone'] = metadata_entry.attrib['value']
    for stat in statistics:
        if stat.attrib['type'] == 'HKQuantityTypeIdentifierActiveEnergyBurned':
            additional_stats['activeCals'] = stat.attrib['sum']
        elif stat.attrib['type'] == 'HKQuantityTypeIdentifierBasalEnergyBurned':
            additional_stats['restingCals'] = stat.attrib['sum']
        elif stat.attrib['type'] == 'HKQuantityTypeIdentifierHeartRate':
            additional_stats['avgHR'] = stat.attrib['average']
            additional_stats['minHR'] = stat.attrib['minimum']
            additional_stats['maxHR'] = stat.attrib['maximum']
    
    final_record = workout_record.attrib | additional_stats
    record_list.append(final_record)

print(record_list)

[{'workoutActivityType': 'HKWorkoutActivityTypeWalking', 'duration': '2835.268574865659', 'durationUnit': 'min', 'sourceName': 'Madeline’s Apple\xa0Watch', 'sourceVersion': '10.2', 'device': '<<HKDevice: 0x303b9c3c0>, name:Apple Watch, manufacturer:Apple Inc., model:Watch, hardware:Watch5,9, software:10.2>', 'creationDate': '2024-01-18 13:10:54 -0400', 'startDate': '2024-01-16 13:55:36 -0400', 'endDate': '2024-01-18 13:10:52 -0400', 'timeZone': 'America/Vancouver', 'restingCals': '991.872', 'avgHR': '80.9773', 'minHR': '72', 'maxHR': '113', 'activeCals': '52.118'}, {'workoutActivityType': 'HKWorkoutActivityTypeYoga', 'duration': '22.553774001201', 'durationUnit': 'min', 'sourceName': 'Madeline’s Apple\xa0Watch', 'sourceVersion': '10.2', 'device': '<<HKDevice: 0x303b9c3c0>, name:Apple Watch, manufacturer:Apple Inc., model:Watch, hardware:Watch5,9, software:10.2>', 'creationDate': '2024-01-18 13:34:17 -0400', 'startDate': '2024-01-18 13:11:41 -0400', 'endDate': '2024-01-18 13:34:14 -0400

In [234]:
records = pd.DataFrame(record_list)
# remove long names

records['workoutActivityType']= records['workoutActivityType'].str.replace('HKWorkoutActivityType','')
# drop unneeded columns
# note that durationUnit is always in mins
records.drop(['sourceName','sourceVersion', 'creationDate','durationUnit', 'device'],axis=1,inplace=True)

# convert dates to datetime
records[['startDate','endDate']] = records[['startDate','endDate']].apply(lambda date:pd.to_datetime(date))

# this record just looked very weird in my set so i deleted it
records.drop(0,inplace=True)

# convert to numerics
records[['duration','restingCals',
         'avgHR','minHR',
         'maxHR','activeCals']] = records[['duration','restingCals',
                                           'avgHR','minHR',
                                           'maxHR','activeCals']].apply(pd.to_numeric)

print(records.head())

           workoutActivityType   duration                 startDate  \
1                         Yoga  22.553774 2024-01-18 13:11:41-04:00   
2                         Yoga  18.617671 2024-01-19 23:33:42-04:00   
3                      Walking   6.748169 2024-01-20 18:18:30-04:00   
4                      Walking  19.577287 2024-01-20 19:46:46-04:00   
5  TraditionalStrengthTraining  52.533711 2024-01-20 20:06:27-04:00   

                    endDate           timeZone  restingCals    avgHR  minHR  \
1 2024-01-18 13:34:14-04:00  America/Vancouver      28.5226  123.170     95   
2 2024-01-19 23:52:19-04:00  America/Vancouver      23.5440  115.669     77   
3 2024-01-20 18:25:15-04:00  America/Vancouver       8.3700  112.120    100   
4 2024-01-20 20:06:21-04:00  America/Vancouver      24.2733  123.970    107   
5 2024-01-20 20:58:59-04:00  America/Vancouver      64.8119  147.323    103   

   maxHR  activeCals  
1    162     99.7136  
2    155     72.1290  
3    140     23.4690  
4    1

In [235]:
'''
    We need to convert the record back to its respective timezone.
    When you move to another time zone, Apple automatically transforms
    your records into the new time zone.

    For example, if you do a workout at 7pm in Vancouver, then two
    months later you move to Toronto, your workout will now show up
    as being at 10pm. 

    This screws up my analytics - I want to know whether I worked
    out at 7pm. I don't care about time zone!

    It should be noted there is a time zone in these records, 
    so the conversion itself is easy. We just need to loop 
    through to figure out what timezone to convert back to.
'''

records['startDate'] = records.apply(lambda x: x['startDate'].tz_convert(x['timeZone']),axis=1)
records['endDate'] = records.apply(lambda x: x['endDate'].tz_convert(x['timeZone']),axis=1)
    
# drop timezone column as we don't need anymore
records = records.drop('timeZone',axis=1)

# we have to convert this to a python datetime
# or else the weekday functions don't work.
records['startDate'] = records['startDate'].apply(lambda date:date.to_pydatetime())
records['endDate'] = records['endDate'].apply(lambda date:date.to_pydatetime())
print(records.head())

           workoutActivityType   duration                  startDate  \
1                         Yoga  22.553774  2024-01-18 09:11:41-08:00   
2                         Yoga  18.617671  2024-01-19 19:33:42-08:00   
3                      Walking   6.748169  2024-01-20 14:18:30-08:00   
4                      Walking  19.577287  2024-01-20 15:46:46-08:00   
5  TraditionalStrengthTraining  52.533711  2024-01-20 16:06:27-08:00   

                     endDate  restingCals    avgHR  minHR  maxHR  activeCals  
1  2024-01-18 09:34:14-08:00      28.5226  123.170     95    162     99.7136  
2  2024-01-19 19:52:19-08:00      23.5440  115.669     77    155     72.1290  
3  2024-01-20 14:25:15-08:00       8.3700  112.120    100    140     23.4690  
4  2024-01-20 16:06:21-08:00      24.2733  123.970    107    149     62.7400  
5  2024-01-20 16:58:59-08:00      64.8119  147.323    103    189    327.4610  


In [236]:
# WHO WANTS FEATURE ENGINEERING?
# I DOOOOO!!!!! MEMEMEMEMEME!!!!!! MEEEEEE!!!!!

# create total cal column
records['totalCals'] = records['restingCals'] + records['activeCals']

# create weekday column - note it is based on the start date.
# the end date MAY be another day because I track my clubbing
# as a social dance on my apple watch. but i dont think that's relevant
records['weekDay'] = records['startDate'].apply(lambda x:x.strftime("%A"))

# split hours and minutes into its own column
records['startHour'] = records['startDate'].apply(lambda x:x.strftime('%H'))
records['startMinutes'] = records['startDate'].apply(lambda x:x.strftime('%M'))
records['endHour'] = records['endDate'].apply(lambda x:x.strftime('%H'))
records['endMinutes'] = records['endDate'].apply(lambda x:x.strftime('%M'))

# clean the start & end date columns to only show dates now that we have the hour+minutes
records['startDate'] = records['startDate'].apply(lambda x:x.strftime('%Y-%m-%d'))
records['endDate'] = records['endDate'].apply(lambda x:x.strftime('%Y-%m-%d'))

print(records.head())

           workoutActivityType   duration   startDate     endDate  \
1                         Yoga  22.553774  2024-01-18  2024-01-18   
2                         Yoga  18.617671  2024-01-19  2024-01-19   
3                      Walking   6.748169  2024-01-20  2024-01-20   
4                      Walking  19.577287  2024-01-20  2024-01-20   
5  TraditionalStrengthTraining  52.533711  2024-01-20  2024-01-20   

   restingCals    avgHR  minHR  maxHR  activeCals  totalCals   weekDay  \
1      28.5226  123.170     95    162     99.7136   128.2362  Thursday   
2      23.5440  115.669     77    155     72.1290    95.6730    Friday   
3       8.3700  112.120    100    140     23.4690    31.8390  Saturday   
4      24.2733  123.970    107    149     62.7400    87.0133  Saturday   
5      64.8119  147.323    103    189    327.4610   392.2729  Saturday   

  startHour startMinutes endHour endMinutes  
1        09           11      09         34  
2        19           33      19         52  
3 